In [1]:
import pandas as pd
import requests
import os
from pathlib import Path

In [ ]:
# Função para listar os arquivos no diretório do repositório
def get_files_in_github_repo(owner, repo, path):
    url = f"https://api.github.com/repos/{owner}/{repo}/contents/{path}"
    response = requests.get(url)

    if response.status_code == 200:
        files = response.json()
        return [file['name'] for file in files]
    else:
        print(f"Erro ao acessar o repositório: {response.status_code}")
        return []

# Função para gerar a URL de um arquivo CSV
def get_url(owner, repo, path, filename):
    return f"https://raw.githubusercontent.com/{owner}/{repo}/main/{path}/{filename}"

In [ ]:
# Informações do repositório
owner = "ericarfs"
repo = "ETL-Netflix"
path = "src/data/raw"

# Listar arquivos no repositório
files = get_files_in_github_repo(owner, repo, path)

# Ler cada arquivo e armazenar os DataFrames em um dicionário
excel_files = []
for file in files:
    url = get_url(owner, repo, path, file)
    excel_files.append(url)

In [ ]:
if not excel_files:
  print("Nenhum arquivo compátivel encontrado")
else:
  dfs = []
  for file in excel_files:
    try:
      #ler arquivo do excel
      df_temp = pd.read_excel(file)

      #pegar o nome do arquivo
      file_path = Path(os.path.basename(file))

      # Obtém o nome do arquivo sem a extensão
      file_name = file_path.stem
      df_temp["location"] = file_name.split("_")[2][:2]

      #obter nome da campanha
      df_temp["campaign"] =  df_temp["utm_link"].str.extract(r'utm_campaign=(.*)')

      #Guardar dados tratados dentro de uma dataframe
      dfs.append(df_temp)

    except Exception as e:
      print(f"Erro ao ler o arquivo {file}: {e}")


In [ ]:
BASE_DIR = "./"
outputpath = "src/data/ready"
if dfs:
  result = pd.concat(dfs, ignore_index=True)
  result = result.sort_values(by="sale_date", ascending=True)
  result = result.drop_duplicates()
  output_file = os.path.join(BASE_DIR, "clean.xlsx")
  result.to_excel(output_file, index=False)